<a href="https://colab.research.google.com/github/fedeholm/Simplifying-a-Complex-Textual-Landscape/blob/main/web_crawler_unified_agenda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import re
from collections import Counter

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [4]:
# import a csv file with hyperlinks ready to be read by a web crawler

# Load the CSV file into a pandas DataFrame
try:
    data = pd.read_csv('Spring2020_Reg_agenda_links.csv', encoding='latin-1')
    print("CSV file loaded successfully.")
    print("First 5 rows of the DataFrame:")
    print(data.head())

    # Now, the DataFrame 'df' is ready to be used by your web crawler function
    # You can call your existing function with this DataFrame
    # processed_df, total_deregulatory_count = crawl_and_count_deregulatory(df, url_column='Link')

    # print("\nDataFrame with counts:")
    # print(processed_df)

    # print(f"\nTotal occurrences of 'deregulatory' across all links: {total_deregulatory_count}")

except UnicodeDecodeError:
    print("Error: Could not decode the file using 'latin-1' encoding. Please try another encoding like 'cp1252'.")
except FileNotFoundError:
    print("Error: 'Reg_agenda_links.csv' not found. Please make sure the file is in the correct directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

CSV file loaded successfully.
First 5 rows of the DataFrame:
      Agency Agenda Stage of Rulemaking  \
0   USDA/OGC           Final Rule Stage   
1  USDA/NIFA        Proposed Rule Stage   
2   USDA/FSA           Final Rule Stage   
3   USDA/FSA           Final Rule Stage   
4   USDA/FSA           Final Rule Stage   

                                               Title        RIN  \
0  Department of Agriculture Civil Monetary Penal...  0510-AA04   
1  Matching Funds Requirement for Capacity Funds ...  0524-AA68   
2  Debt Collection, Debt Settlement, and Assignme...  0560-AH09   
3            Acreage Reporting and Common Provisions  0560-AH69   
4  Direct Farm Ownership Loan (FO) Microloan (ML)...  0560-AI33   

                                                Link  \
0  https://www.reginfo.gov/public/do/eAgendaViewR...   
1  https://www.reginfo.gov/public/do/eAgendaViewR...   
2  https://www.reginfo.gov/public/do/eAgendaViewR...   
3  https://www.reginfo.gov/public/do/eAgendaViewR... 

In [5]:
# Webcrawler to extract designations

# Load the CSV file into a pandas DataFrame

try:
    data_links = pd.read_csv('Spring2019_Reg_agenda_links.csv', encoding='latin-1')
except UnicodeDecodeError:
    print("Error: Could not decode the file using 'latin-1' encoding. Please try another encoding like 'cp1252'.")
except FileNotFoundError:
    print("Error: 'Reg_agenda_links.csv' not found. Please make sure the file is in the correct directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


links_df = pd.DataFrame(data_links)


# web crawler to capture the following 2 strings after "EO 13771 Designation" in the links explored

def crawl_and_extract_designations(df, url_column='Link'):
    """
    Crawls each URL in a DataFrame column, extracts the two strings following
    "EO 13771 Designation", and returns a new DataFrame with the extracted strings.

    Args:
        df (pd.DataFrame): The input DataFrame containing URLs.
        url_column (str): The name of the column in the DataFrame containing URLs.

    Returns:
        pd.DataFrame: A new DataFrame with added 'Designation_1' and 'Designation_2' columns.
    """
    if url_column not in df.columns:
        raise ValueError(f"DataFrame does not have a column named '{url_column}'")

    # Initialize columns to store the extracted strings
    #df['Designation_1'] = None
    df['Designation_2'] = None

    search_pattern = re.compile(r'EO 13771 Designation:\s*(.*?)\s*(.*)')

    for index, row in df.iterrows():
        url = row[url_column]
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text()

            # Search for the pattern in the text
            match = search_pattern.search(text)

            if match:
                designation1 = match.group(1).strip()
                designation2 = match.group(2).strip()
                #df.loc[index, 'Designation_1'] = designation1
                df.loc[index, 'Designation_2'] = designation2
                print(f"Processed {url}: Abstract: '{designation2}'")
            else:
                print(f"Processed {url}: 'EO 13771 Designation' not found or pattern mismatch.")

        except requests.exceptions.RequestException as e:
            print(f"Error processing {url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {url}: {e}")

    return df

# Example Usage with the sample DataFrame defined previously:
processed_df_designations = crawl_and_extract_designations(links_df, url_column='Link')

#print("\nDataFrame with extracted designations:")
#processed_df_designations

Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=201904&RIN=0503-AA61: Abstract: 'Not subject to, not significant'
Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=201904&RIN=0524-AA71: Abstract: 'Not subject to, not significant'
Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=201904&RIN=0551-AA91: Abstract: 'Not subject to, not significant'
Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=201904&RIN=0551-AA93: Abstract: 'Not subject to, not significant'
Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=201904&RIN=0551-AA94: Abstract: 'Not subject to, not significant'
Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=201904&RIN=0551-AA95: Abstract: 'Not subject to, not significant'
Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=201904&RIN=0551-AA96: Abstract: 'Not subject to, not significant'
Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=201904&RIN

In [ ]:
# export the new DataFrame to CSV
processed_df_designations.to_csv('processed_output_designations.csv', index=False)

In [8]:
############# IF NEEDED - LOAD A NEW VERSION OF THE DF

# Load the CSV file into a pandas DataFrame

try:
    data_links = pd.read_csv('ONLY_ABSTRACT_NEEDED_Fall2020_Reg_agenda_links.csv', encoding='latin-1')
except UnicodeDecodeError:
    print("Error: Could not decode the file using 'latin-1' encoding. Please try another encoding like 'cp1252'.")
except FileNotFoundError:
    print("Error: 'Reg_agenda_links.csv' not found. Please make sure the file is in the correct directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


links_df = pd.DataFrame(data_links)

In [10]:
# Webcrawler to extract abstracts

# web crawler to capture the following 2 strings after "Abstract" in the links
# explored

def crawl_and_extract_designations(df, url_column='URL'):
    """
    Crawls each URL in a DataFrame column, extracts the two strings following
    "Abstract", and returns a new DataFrame with the extracted strings.

    Args:
        df (pd.DataFrame): The input DataFrame containing URLs.
        url_column (str): The name of the column in the DataFrame containing URLs.

    Returns:
        pd.DataFrame: A new DataFrame with added 'Abstract_1' column.
    """
    if url_column not in df.columns:
        raise ValueError(f"DataFrame does not have a column named '{url_column}'")

    # Initialize columns to store the extracted strings
    #df['Designation_1'] = None
    df['Abstract_2'] = None

    search_pattern = re.compile(r'Abstract:\s*(.*?)\s*(.*)')

    for index, row in df.iterrows():
        url = row[url_column]
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text()

            # Search for the pattern in the text
            match = search_pattern.search(text)

            if match:
                abstract2 = match.group(2).strip()
                df.loc[index, 'Abstract_2'] = abstract2
                print(f"Processed {url}: Abstract: '{abstract2}'")
            else:
                print(f"Processed {url}: 'Abstract' not found or pattern mismatch.")

        except requests.exceptions.RequestException as e:
            print(f"Error processing {url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {url}: {e}")

    return df

# Example Usage with the sample DataFrame defined previously:
processed_df_abstracts = crawl_and_extract_designations(links_df, url_column='URL')



Processed https://www.reginfo.gov/public/do/eAgendaViewRule?pubId=202010&RIN=0503-AA67: Abstract: 'Pursuant to multiple cross-agency Administration initiatives, including but not limited to Executive Orders 13771, 13859, and the President’s Management Agenda, and longstanding commitment to robust retrospective review of federal regulations, this rule will reflect use of Artificial Intelligence (AI) and other advanced analytical tools to conduct a thorough analysis of the agency’s operative regulatory code in order to identify opportunities for administrative burden reduction. This may include reconciling conflicting, eliminating duplicative, or otherwise streamlining reporting requirements and other administrative burdens. It may also include corrections of typographical errors or broken and obsolete citations. Whereas this kind of a comprehensive code cleanup effort would previously required a costly and time-consuming manual effort, use of new technologies, such as natural language p

In [11]:
# export the new DataFrame to CSV
processed_df_abstracts.to_csv('FALL_2020_processed_abstracts_designations.csv', index=False)

Next we import the necessary packages and start coding our text classifier.

In [12]:
# !pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

In [23]:
df = pd.read_csv("TrainTest_designations_all_years.csv")
X = df['Abstract']
y = df['Designation']

In [24]:
df['Designation'].value_counts()

,count
Designation,
Deregulatory,3405
Regulatory,1240


In [25]:
# splitting our dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=57)


In [26]:
# We will use the scikit-learn pipeline which is just a sequence of steps to take
# to build our classifier. In this pipeline, we convert the data to a numerical
# format using TfidfVectorizer and then specify our classifiers

pipeMNB = Pipeline([('tfidf', TfidfVectorizer()),('clf', MultinomialNB())])
pipeCNB = Pipeline([('tfidf', TfidfVectorizer()),('clf', ComplementNB())])
pipeSVC = Pipeline([('tfidf', TfidfVectorizer()),('clf', LinearSVC())])

In [37]:
# We can try to optimize the best performing models with additional arguments

pipeMNB = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,3))),('clf', MultinomialNB())])
pipeCNB = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,3))),('clf', ComplementNB())])
pipeSVC = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,3))),('clf', LinearSVC())])

In [34]:
# Now we will build our models using MultinomialNB, ComplementNB, and LinearSVC,
# and training it (fitting) on our train data. After that, we predict the labels
# for our TEST data, and then we print out the accuracy score based on a
# comparison of the correct labels and our predictions

pipeMNB.fit(X_train, y_train)
predictMNB = pipeMNB.predict(X_test)
print(f"MNB: {accuracy_score(y_test, predictMNB):.2f}")

pipeCNB.fit(X_train, y_train)
predictCNB = pipeCNB.predict(X_test)
print(f"CNB: {accuracy_score(y_test, predictCNB):.2f}")


MNB: 0.88
CNB: 0.93
SVC: 0.95


In [38]:
# best performing so far
pipeSVC.fit(X_train, y_train)
predictSVC = pipeSVC.predict(X_test)
print(f"SVC: {accuracy_score(y_test, predictSVC):.2f}")

SVC: 0.95


In [45]:
# we can see how the classifeir performs in a new piece of text

# noticed that 'new' 'updated' is attached to 'dereg'.

abstract = "This rule will safeguard health of outdoor workers"
result = pipeSVC.predict([abstract])
print("Result: ", result[0])

Result:  Regulatory
